# Select asteroid comparisons; JPL Horizon vs OpenOrb, OrbFit

## Assumptions/ documentation:

* Values for orbit definition taken from JPL Horizons elements
* Observing Location: I11, Gemini South
* Integration Time: 2010/01/01 to 2020/01/01, midnights

## Current questions/ issues:
* OrbFit 60 arcsecond bug
* OrbFit Covariance matrix effect/ workaround
* Documenting functions (work in progress)
* Find_Orb takes state vectors, also not available in COM

## Info Pointers
* table color-coding: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
* Keep sign separate from columns: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_fwf.html, https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#csv-text-files
* OrbFit questions via email thread w/ Federica
* Find_Orb questions in Slack

# Imports and Initialization

In [1]:
import os
import numpy as np
import pandas as pd
import textwrap
ddent = textwrap.dedent
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import QTable
from astropy.io import ascii
import astropy as ap

from astroquery.jplhorizons import Horizons
import pyoorb as oo
import tempfile as tf
oo.pyoorb.oorb_init()

0

In [2]:
integ_functions = {}

# JPL Horizons- Elements and Integrator

In [3]:
# calculates elements
def get_elems(obj_id, start):
    '''
    Parameters
    ----------
    obj_id : `str`
        Number of asteroid, defined by MPC.
    start : `str`
        Beginning time of integration, UTC.
        
    Returns
    -------
    el_jpl : `~numpy.ndarray` (N, 12)
        Orbital elements as returned by jpl, first column is obj_id.   
    '''
    
    epochs = ap.time.Time(start).jd
    
    el_obj = Horizons(id=obj_id, location= '500@10',
               epochs=epochs)
    el_jpl = el_obj.elements()    
    
    el_jpl['targetname'] = obj_id
    
    return el_jpl

In [4]:
# calculates ephemerides for object via JPL Horizons
def get_ephem_jpl(el_jpl, start, stop, obs):
    '''
    Parameters
    ----------
    el_jpl : `~numpy.ndarray` (N, 12)
        Orbital elements as returned by jpl.    
    start : `str`
        Beginning time of integration, UTC.   
    stop : `str`
        End time of integration, UTC.    
    obs : `str`
        Observatory code.
        
    Returns
    -------
    coord_jpl : `~numpy.ndarray` (N, 2)
        RA and DEC coordinates of ephemderides as determined by JPL, units deg
            RA : first column of coord_jpl
            DEC : second column of coord_jpl     
    mag_jpl : `~numpy.ndarray` (N, 1)
        Contains magnitudes of object as determined by JPL, units mags
    '''
    
    obj_id = el_jpl['targetname'][0]

    ephem_obj = Horizons(id= obj_id, location= obs,
               epochs={'start': start, 'stop':stop,
                      'step':'1d'})
    ephem_jpl = ephem_obj.ephemerides()

    coord_jpl = np.array([ephem_jpl['RA'], ephem_jpl['DEC']]) * u.deg
    mag_jpl = np.array([ephem_jpl['V']]) * u.mag

    return coord_jpl, mag_jpl

integ_functions["JPL Horizons"] = get_ephem_jpl

# OpenOrb/ pyoorb- Input formating and Integrator

In [5]:
# Reorganizes JPL Horizons elements output into pyoorb-acceptable input. (Expand for multiple orbits next?)
def pyoorb_input(orbits, epoch):
    '''
    Parameters
    ----------
    orbits : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
    epoch : `~numpy.ndarray` (3652, 2)
        Constrained to cometary format.
        
    Returns
    -------
    new_array : `~numpy.ndarray` (N, 12)
        Orbits formatted in the format expected by PYOORB. 
            orbit_id : index of input orbits
            elements x6: orbital elements of propagated orbits
            orbit_type : orbit type
            epoch_mjd : epoch of the propagate orbit
            time_scale : time scale of output epochs
            H/M1 : absolute magnitude
            G/K1 : photometric slope parameter
    '''
    
    temp = orbits.copy()
    temp = temp.as_array().data
    if temp.shape == (6,):
        num_orbits = 1
    else:
        num_orbits = temp.shape[0]
        
    for i in range(num_orbits):
        ids = i
        orbit_type = 2
        time_scale = 1
        
    # elements x6
    q = temp[0][6]
    e = temp[0][5]
    incl = np.deg2rad(temp[0][7])
    longnode = np.deg2rad(temp[0][8])
    argper = np.deg2rad(temp[0][9])
    peri_epoch = ap.time.Time(temp[0][10], format='jd').mjd

    mag = temp[0][3]
    slope = temp[0][4]
    
    if num_orbits > 1:
        new_array = np.array(
            np.array([
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]), 
            dtype=np.double, 
            order='F')
    else:
        new_array = np.array(
            [[
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]], 
            dtype=np.double,
            order='F')
    
    return new_array

In [6]:
# Calculates ephemerides using PYOORB
def get_ephem_OpenOrb(el_jpl, start, stop, obs):
    '''
    Parameters
    ----------
    el_jpl : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
    start : `str`
        Beginning time of integration, UTC.   
    stop : `str`
        End time of integration, UTC.
    obs : `str`
        Observatory code.
        
    Returns
    ------- 
    coord_OpenOrb : `~numpy.ndarray` (N, 2)
        RA and DEC coordinates of ephemderides as determined by PYOORB, units deg
            RA : first column of coord_OpenOrb
            DEC : second column of coord_OpenOrb 
            
    mag_OpenOrb :  `~numpy.ndarray` (N, 1)
        Contains magnitudes of object as determined by PYOORB, units mags
    '''
    
    # time conversions, epochs for pyoorb to work
    element_time_pyoorb = ap.time.Time(start).mjd
    start_pyoorb = ap.time.Time(start).mjd
    stop_pyoorb = ap.time.Time(stop).mjd
    peri_time = ap.time.Time(el_jpl['Tp_jd'][0], format='jd').mjd
    
    #conversion and implementation
    pyoorb_formatted = pyoorb_input(el_jpl, start_pyoorb)
    t0 = np.array([element_time_pyoorb, 1], dtype=np.double, order='F')
    mjds = np.arange(start_pyoorb, stop_pyoorb, 1)
    epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
    ephem_pyoorb, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=pyoorb_formatted,
                                         in_obscode=obs,
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')
    if err != 0:
        print(err)
        
    coord_OpenOrb = np.array([ephem_pyoorb[0][:,1],ephem_pyoorb[0][:,2]]) * u.deg
    mag_OpenOrb = np.array([ephem_pyoorb[0][:,9]]) * u.mag
    
    return coord_OpenOrb, mag_OpenOrb

integ_functions["OpenOrb"] = get_ephem_OpenOrb

# OrbFit- Input formatting and Integrator

In [7]:
# Creates .eq1 file in epoch folder for OrbFit
def eq1file(el_jpl, tdir):
    '''
    Parameters
    ----------
    el_jpl : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
    tdir : `tf.TemporaryDirectory`
        Temporary directory object, temp directory for file generation
        
    Returns
    ------- 
    temp.eq1: temporary text file to run OrbFit
    '''  
    
    os.makedirs(f'{tdir}/epoch')
    
    with open(f"{tdir}/epoch/temp.eq1", "w") as fp:
        fp.write(ddent(f'''\
        format  = 'OEF2.0'       ! file format
        rectype = 'ML'           ! record type (1L/ML)
        refsys  = ECLM J2000     ! default reference system
        END_OF_HEADER
        temp
        ! Cometary elements: q, e, i, long. node, arg. peric., pericenter time\n'''))
        fp.write(" COM   %.15E  %.15f   %.15f  %.15f  %.15f   %.15f\n" % (el_jpl['q'][0], el_jpl['e'][0], el_jpl['incl'][0], el_jpl['Omega'][0], el_jpl['w'][0], ap.time.Time(el_jpl['Tp_jd'][0], format='jd').mjd))
        fp.write(" MJD     %.9f TDT\n" % (ap.time.Time(el_jpl['datetime_jd'][0], format='jd').mjd))
        fp.write(" MAG  %.3f  %.3f\n" % (el_jpl['H'][0], el_jpl['G'][0]))
        fp.write(ddent(f'''\
        ! Non-grav parameters: model used, actual number in use, dimension
         LSP   0  0    6
        ! RMS    1.68232E-08   5.88900E-08   8.22688E-08   5.34469E-08   7.56890E-08   7.13398E-06
        ! EIG   1.07579E-08   5.31009E-08   5.91864E-08   7.57356E-08   7.86169E-08   1.27486E-07
        ! WEA   0.08651  -0.02289  -0.24412  -0.03357  -0.02078  -0.96480
         COV   2.830210138102556E-16  3.543122024213312E-16 -2.603292682702056E-16
         COV  -4.992042214900484E-18 -1.052690180196314E-18 -7.873861865190710E-14
         COV   3.468027286557968E-15  6.878077752471183E-17  1.886511729787680E-17
         COV   6.689670038864485E-17  1.808279351538482E-14  6.768149177265766E-15
         COV   7.159243161286040E-17  1.248926483233068E-16  1.357728291186093E-13
         COV   2.856568560436748E-15  2.588049637167598E-16  2.529981071526617E-14
         COV   5.728829671236270E-15  1.056596023015451E-14  5.089368128905899E-11
         NOR   8.462990106959648E+15 -9.345934921051774E+14  6.961302078833404E+13
         NOR  -9.766026206616650E+13 -9.148695418092123E+12  1.329006055003970E+13
         NOR   3.921580648140324E+14 -8.566206317904612E+12  1.006265833999790E+13
         NOR  -2.128841368531985E+12 -1.566971456817283E+12  1.567202493495569E+14
         NOR  -7.910041612493922E+11 -2.702958007388599E+12 -3.063965034542373E+11
         NOR   3.541407046591562E+14 -1.551670529664669E+13 -3.253830675316872E+11
         NOR   1.754031538722264E+14 -3.488851624201696E+10  4.175326401599722E+10
         '''))
    
    return

In [8]:
# Writes .fop file for OrbFit
def fopfile(tdir):
    '''
    Parameters
    ----------
    tdir : `tf.TemporaryDirectory`
        Temporary directory object, temp directory for file generation
    
    Returns
    ------- 
    temp.fop: temporary text file to run OrbFit
    '''
        
    with open(f"{tdir}/temp.fop", "w") as fp:
        fp.write(ddent('''\
        ! input file for fitobs
        fitobs.
        ! first arc        .astna0='temp'           ! full name, first arc
                .obsdir0='mpcobs/'         ! directory of observs. file, first arc
                .elefi0='epoch/temp.eq1' ! first arc elements file

        ! second arc
        !        .astnap=''            ! full name, second arc
        !        .obsdirp='mpcobs'     ! directory of observs. file, second arc
        ! bizarre  control;
                .ecclim=     1.9999d0    ! max eccentricity for non bizarre orbit
                .samin=      0.3d0       ! min a for non bizarre orbit
                .samax=      2000.d0     ! max a for non bizarre orbit
                .phmin=      0.001d0     ! min q for non bizarre orbit
                .ahmax=     4000.d0      ! max Q for non bizarre orbit
                .error_model='fcct14'     ! error model
        propag.
                .iast=17            ! 0=no asteroids with mass, n=no. of massive asteroids (def=0)
                .filbe='AST17'      ! name of the asteroid ephemerides file (def='CPV')
                .npoint=600         ! minimum number of data points for a deep close appr (def=100)
                .dmea=0.2d0         ! min. distance for control of close-app. to the Earth only (def=0.1)
                .dter=0.05d0        ! min. distance for control of close-app.
                                    ! to terrestrial planets (MVM)(def=0.1)
                .yark_exp=2.d0      ! A2/r^yark_exp model (def=2)
                .ngr_opt=.TRUE.     ! read options for non-gravitational perturbations from the option file
                .irel=2             ! 0=newtonian 1=gen. relativity, sun 2=gen. rel. all planets
                                    !          (def=0, 1 for NEA, 2 for radar)
                .iaber=2            ! aberration 0=no 1=yes 2=(def=1)
                .ilun=1             ! 0=no moon 1= yes (def=0, 1 for NEA)
                .iyark=3            ! 0=no Yarkovsky, 1=Yark diurnal, 2=Yark seasonal
                                    !    3=secular nongravitational perturbations (including Yark) (def=0)
                .ipa2m=0           ! 0=no drpa2m, 1=yes spherical direct radiation pressure (def=0)
                .det_drp=2          ! how many parameters to solve: 0=none 1=drpa2m 2=dadt 3=both (def=0)
                .det_outgas=0       ! det outgassing for comets

        difcor.

        IERS.
                .extrapolation=.T. ! extrapolation of Earth rotation

        reject.
                .rejopp=.false.    ! reject entire opposition
        '''))
        
    import shutil
    shutil.copyfile("2019OK/AST17.bai", f"{tdir}/AST17.bai")
    shutil.copyfile("2019OK/AST17.bep", f"{tdir}/AST17.bep")

    return

In [9]:
# Writes file to bypass OrbFit's interactive menu
def astfile(start, stop, obs, tdir):
    '''
    Parameters
    ----------
    start : `str`
        Beginning time of integration, UTC.  
    stop : `str`
        End time of integration, UTC.   
    obs : `str`
        Observatory code.
    tdir : `tf.TemporaryDirectory`
        Temporary directory object, temp directory for file generation
        
    Returns
    ------- 
    ast.inp: text file required to bypass OrbFit interactive menu
    '''
    
    with open(f"{tdir}/ast.inp", "w") as fp:
        fp.write(ddent(f'''\
        temp
        6
        6
        {ap.time.Time(start).mjd}
        {ap.time.Time(stop).mjd}
        1
        {obs}
        0
        '''))
        
    return

In [10]:
# Calculates ephemerides using OrbFit
def get_ephem_OrbFit(el_jpl, start, stop, obs):
    '''
    Parameters
    ----------        
    el_jpl : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
    start : `str`
        Beginning time of integration, UTC. 
    stop : `str`
        End time of integration, UTC.  
    obs : `str`
        Observatory code.
        
    Returns
    -------
    coord_OrbFit : `~numpy.ndarray` (N, 2)
        RA and DEC coordinates of ephemderides as determined by OpenOrb, units deg
            RA : first column of coord_OrbFit
            DEC : second column of coord_OrbFit 
            
    mag_OrbFit : `~numpy.ndarray` (N, 1)
        Contains magnitudes of object as determined by OpenOrb, units mags
    '''
    
    with tf.TemporaryDirectory() as tdir:
        eq1file(el_jpl, tdir=tdir)
        fopfile(tdir=tdir)
        astfile(start, stop, obs, tdir=tdir)
            # cd into temp directory, run things there
        home = os.environ["HOME"]
        ! (cd "{tdir}" && {home}/great-integrator-bake-off/orbfit/src/fitobs/fitobs.x < ast.inp) > /dev/null #inside parenth when fixed
        df = pd.read_fwf(f'{tdir}/temp.eph', skiprows=4, header=None, colspecs=[(20,32),(35,37),(38,40),(41,47),(49,50),(50, 52),(53, 55),(56, 61),(62,67)])

    df["RA"] = Angle((df[1], df[2], df[3]), unit = 'hourangle').degree
    df["DEC"] = Angle((df[5], df[6], df[7]), unit = u.deg)
    df.loc[df[4] == '-', "DEC"] *= -1
    df['Mag'] = df[8]

    coord_OrbFit = np.array([df["RA"], df["DEC"]]) * u.deg
    mag_OrbFit = np.array([df['Mag']]) * u.mag
    
    return coord_OrbFit, mag_OrbFit

integ_functions["OrbFit"] = get_ephem_OrbFit

# FindOrb- Input formatting and Integrator

In [11]:
def get_ephem_FindOrb():
    !./fo '2019OK/temp.eq1' -ephem
    
    return coord_FindOrb

# Outputs and comparison

In [12]:
# Takes in object id and calls all integrators
def get_ephems(obj_id, start, stop, obs):
    '''
    Parameters
    ----------
    obj_id : `str`
        Number of asteroid, defined by MPC.
    start : `str`
        Beginning time of integration, UTC.   
    stop : `str`
        End time of integration, UTC.   
    obs : `str`
        Observatory code.
        
    Returns
    -------
    el_jpl : `~numpy.ndarray` (N, 18)
        Orbital elements as determined by JPL Horizons.
    results : `dict` (N, size(integ_functions), 3)
        (RA, DEC) units deg + (mag) units mags as determined by all integrators in integ_functions
        '''
    
    el_jpl = get_elems(obj_id, start)
    print("Elements done")
    results = {}
    
    for name, func in integ_functions.items():
        #if name in ["OpenOrb", "JPL Horizons"]:
            #continue
        results[name] = func(el_jpl, start, stop, obs)
        print(f"{name} done")
        
    return el_jpl, results

## Difference metric

In [13]:
# Maximum circle difference function. Calculates great circle difference and other difference metrics (future)
def gc_dist(results, reference="JPL Horizons"):
    '''
    Parameters
    ----------
    results : `dict` (N, size(integ_functions), 3)
        (RA, DEC) units deg + (mag) units mags as determined by all integrators in integ_functions
    reference : `str` kwarg
        Keyword argument with reference to which integrator all computations are made. 
        Current default is set as JPL Horizons.
        
    Returns
    -------
    rsep : `dict` (N, size(integ_functions))
        Difference of coordinates as calculated by SkyCoord from astropy.
    rmag : `dict` (N, size(integ_functions))
        Difference of magnitudes.
        '''

    ref = results[reference]
    coord_ref = SkyCoord(ref[0][0], ref[0][1], frame='icrs', unit="deg")
    
    rsep, rmag = {}, {}
    for integ_name, result in results.items():
        if integ_name == reference:
                continue

        # compute the coordinate difference relative to a reference integrator
        coord = SkyCoord(result[0][0], result[0][1], frame='icrs', unit="deg")
        sep = coord_ref.separation(coord).arcsec
        rsep[integ_name] = sep
    
        # compute the magnitude difference
        dmag = result[1] - ref[1]
        rmag[integ_name] = dmag

    return rsep, rmag

## Statistic computation and graphical representation

In [14]:
def table_stats(dictionary):
    '''
    Parameters
    ----------
    dictionary : `dict`
        Dictionary of separations or magnitude differences to compute statistics on
    
    Returns
    -------
    table.stack() : `pd.Dataframe`
        pandas DataFrame as a series generated using function
    '''
    
    statistics = pd.DataFrame(data= None, columns=['integrator', 'mean', 'median', 'max'])

    for integ_name, result in dictionary.items():
        statistics = statistics.append({'integrator' : integ_name, 'mean': np.mean(result), 'median': np.median(result), 
                              'max': np.max(result)}, ignore_index = True)
        
    statistics.set_index('integrator', inplace=True)  
    
    return statistics.stack()

In [15]:
# Creates color coding map for tables
def color_map(s):
    ret = []
    for val in s:
        if val < 0.05:
            style = ['background-color: green']
        elif val < 0.2:
            style = ['background-color: yellow']
        elif val < 0.6:
            style = ['background-color: orange']
        else:
            style = ['background-color: red']
        
        ret += style
    return ret

# Example results with select asteroids

In [20]:
result = {}

asteroids = [('MBA','202930'), ('MBA', '110'), ('MBA', '887'), ('MBA', '6489'), ('MBA', '176014'),
             ('KBO','136199'), ('KBO', '15760'), 
             ('PHA', '2018 VP1'), ('PHA', '101955'), ('PHA', '99942'), 
             ('Mars Trojan', '5261'),
             ('Jupiter Trojan', '624'), ('Jupiter Trojan', '588'),
             ('Centaur', '10199'),
             ('TNO', '136472'), ('TNO', '136199'),
             ('Impactor', '2008 TC3'),
             ('NEO', '2020 VT4'), ('NEO', '433'), ('NEO', '367943'),
             ('a Vatira', '2020 AV2'),
             ('Interstellar', 'A/2017 U1')
            ]

start = '2010-01-01T00:00:00'
stop = '2020-01-01T00:00:01'
obs = 'I11'

for kind, obj_id in asteroids:
    el_jpl, results = get_ephems(obj_id, start, stop, obs)
    rsep, rmag = gc_dist(results, reference="JPL Horizons")
    stats = table_stats(rsep)
    result[(kind, obj_id)] = stats

Elements done
JPL Horizons done
OpenOrb done


1       53.91
2       32.62
3        6.69
4       36.05
        ...  
3648    56.16
3649     8.19
3650    15.92
3651    19.34
3652    18.44
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done


1       26.21
2       21.23
3       10.25
4       53.30
        ...  
3648    24.12
3649     1.42
3650    36.01
3651     7.90
3652    37.10
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done


...

OrbFit done
Elements done
JPL Horizons done
OpenOrb done


1       42.11
2       14.98
3       48.25
4       21.91
        ...  
3648    54.10
3649    18.41
3650    43.25
3651     8.61
3652    34.48
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done


1       31.71
2       38.45
3       32.44
4       13.69
        ...  
3648    54.63
3649    19.03
3650    33.24
3651    37.21
3652    30.87
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done


In [21]:
maps = [color_map, color_map_bad, kawaii, kawaii2, goose]

for i in maps:
    display(pd.DataFrame(result).T.style.apply(i))

# Color Map options :)

Let's adopt these:

value <= 0.05" -> green (good)

value <= 0.2" -> yellow (ok)

value <= 0.6" -> orange (not good but may work)

value is red otherwise (bad)

similar color maps below

In [16]:
# Creates color coding map for tables
def color_map_bad(s):
    ret = []
    for val in s:
        if val < 0.05:
            style = ['background-color: #005377']
        elif val < 0.2:
            style = ['background-color: #06a77d']
        elif val < 0.6:
            style = ['background-color: #d5c67a']
        else:
            style = ['background-color: #f1a208']
        
        ret += style
    return ret

In [17]:
def goose(s):
    ret = []
    for val in s:
        if val < 0.05:
            style = ['background-color: #51BBFE']
        elif val < 0.2:
            style = ['background-color: #8FF7A7']
        elif val < 0.6:
            style = ['background-color: #F4E76E']
        else:
            style = ['background-color: #F7FE72']
        
        ret += style
    return ret

In [18]:
def kawaii(s):
    ret = []
    for val in s:
        if val < 0.05:
            style = ['background-color: #DBB1BC'] #D3C4E3
        elif val < 0.2:
            style = ['background-color: #D3C4E3']
        elif val < 0.6:
            style = ['background-color: #8F95D3']
        else:
            style = ['background-color: #58504A']
        
        ret += style
    return ret

In [19]:
def kawaii2(s):
    ret = []
    for val in s:
        if val < 0.05:
            style = ['background-color: #F3FFE1']
        elif val < 0.2:
            style = ['background-color: #FFD7D5']
        elif val < 0.6:
            style = ['background-color: #FFD3E8']
        else:
            style = ['background-color: #E8D7FF']
        
        ret += style
    return ret

# Prep Zone

## Tried it with Ceres no1, Vesta no4,  Lydia no110 (more MBA population)- 1 and 4 refuse to work with 202930 covar.

## Try it with KBO (Eris, no 136199 ; Albion no15760)

## Try a PHA? (2018VP1, 101955, Apophis no99942) 

2020 AV2 (a Vatira), 2020 SO (Earth's mini moon), 367943 Duende, 433 Eros, 
5261 Eureka (Mars Trojan), 

MBA: 887 Alinda, 6489 Golevka, 176014 Vedrana, 

Jupiter Trojans: 624 Hektor, 588 Achilles

Centaur: 10199 Chariklo

Interstellar object 'Oumuamua (A/2017 U1)

TNOs: 136472 Makemake (2005 FY9), 136199 Eris

Earth impactors: 2008 TC3

NEO: 2020 VT4

# Recipe

In [ ]:
'The value of pi is %10.8f slices (and %.3f).' % (3.14159, 456.2)

In [ ]:
'''
Gfortran version 4.8.6

Useful bits from docs:

JPL: 'targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P'

pyoorb:
orbit id: an integer number to identify the orbit; usually ranges from 0 to n-1, where n is the number of orbits
perihelion distance (au) for COM, semimajor axis a (au) for KEP, x (au) for CART
eccentricity for COM or KEP, y (au) for CART
inclination (deg) for COM or KEP, z (au) for CART
longitude of the ascending node (deg) for COM and KEP, dx/dt (au/day) for CART
argument of the periapsis (deg) for COM and KEP, dy/dt (au/day) for CARqT
epoch of perihelion (modified Julian date) for COM, mean anomaly (deg) for KEP, dz/dt for CART
orbital element type; integer value: CART: 1, COM: 2, KEP: 3
epoch of the osculating elements (modified Julian date)
timescale type of the epochs provided; integer value: UTC: 1, UT1: 2, TT: 3, TAI: 4
absolute magnitude of the target
photometric slope parameter of the target


JPL TableColumns values to extract:
[0,1,2,3,7,8,23] = [targetname, datetime str, datettime jd, H, RA, DEC, V]

Want pyoorb.oorb_ephemeris_basic , use these indices for properties:
[0,1,2,9] = [mjd, RA (deg), DEC(deg), predicted apparent V-band mag]

example copied from--> https://github.com/oorb/oorb/tree/master/python#ephemeris-computation

OrbFit bug: 9 60.00 for seconds is wrong

18 May 2013 24.000 56430.999988    6 40  3.478  +24  9 60.00   8.6   6.7    7.933  -26.7  -41.1   58.2   8.4 190.0  2.5146  3.1868    1.0092   -0.0237    0.012"    0.010"  85.4

'''